In [75]:
import os
import numpy as np
import csv
import random
import matplotlib.pyplot as plt

from tslearn.utils import to_time_series_dataset
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
import tslearn.svm as tsvm

In [76]:
# helper functions

def series_append(series, list, keys):
    for i in range(64):
        series[keys[i]].append(float(list[i]))
    return series

def load_series(filename):
    with open(filename, 'r') as csv_in:
        csv_file = list(csv.reader(csv_in))
        series = {}
        keys = csv_file[0]
        for key in keys: series[key] = []
        for i in range(2, len(csv_file), 2):
            series = series_append(series, csv_file[i], keys)
        return [series, int((len(csv_file) - 2) / 2)]

def series_to_arr(series):
    l = []
    for key in series:
        if (key[-1] == 'v'): continue
        l += (series[key])
    return l

def E3(pred, truth):
    num_correct = 0
    for i in range(len(pred)):
        if (abs(pred[i] - truth[i]) <= 3):
            num_correct += 1
    return (100.0 * float(num_correct) / float(len(pred)))

def E2(pred, truth):
    num_correct = 0
    for i in range(len(pred)):
        if (abs(pred[i] - truth[i]) <= 2):
            num_correct += 1
    return (100.0 * float(num_correct) / float(len(pred)))

def E1(pred, truth):
    num_correct = 0
    for i in range(len(pred)):
        if (abs(pred[i] - truth[i]) <= 1):
            num_correct += 1
    return (100.0 * float(num_correct) / float(len(pred)))

def E0(pred, truth):
    num_correct = 0
    for i in range(len(pred)):
        if (abs(pred[i] - truth[i]) == 0):
            num_correct += 1
    return (100.0 * float(num_correct) / float(len(pred)))

In [97]:
# creating datasets

time_normalized_dir = '..\\time_series\\Time_normalized_stages\\1_unprocessed\\'
time_normalized_files = os.listdir(time_normalized_dir)

raw_X_train = []
raw_X_test = []
smoothed_X_train = []
smoothed_X_test = []
translate_X_train = []
translate_X_test = []
final_X_train = []
final_X_test = []

o_label_train = []
o_label_test = []

f_label_train = []
f_label_test = []

b_label_train = []
b_label_test = []

s_label_train = []
s_label_test = []

l_label_train = []
l_label_test = []

while(len(raw_X_test) != 25):
    print(len(raw_X_test))
    raw_X_train = []
    raw_X_test = []
    smoothed_X_train = []
    smoothed_X_test = []
    translate_X_train = []
    translate_X_test = []
    final_X_train = []
    final_X_test = []

    o_label_train = []
    o_label_test = []

    f_label_train = []
    f_label_test = []

    b_label_train = []
    b_label_test = []

    s_label_train = []
    s_label_test = []

    l_label_train = []
    l_label_test = []

    for file in time_normalized_files:
        raw_series, num_frames = load_series('..\\time_series\\Time_normalized_stages\\1_unprocessed\\' + file)
        smoothed_series, num_frames = load_series('..\\time_series\\Time_normalized_stages\\2_smoothed\\' + file)
        translate_series, num_frames = load_series('..\\time_series\\Time_normalized_stages\\3_translation\\' + file)
        final_series, num_frames = load_series('..\\time_series\\Time_normalized_stages\\4_final\\' + file)
        rand = random.randint(0, 100)

        if (rand < 95):

            raw_X_train.append(series_to_arr(raw_series))
            smoothed_X_train.append(series_to_arr(smoothed_series))
            translate_X_train.append(series_to_arr(translate_series))
            final_X_train.append(series_to_arr(final_series))

            o_label_train.append(int(file[4]))
            f_label_train.append(int(file[7]))
            b_label_train.append(int(file[10]))
            s_label_train.append(int(file[13]))
            l_label_train.append(int(file[16]))
        elif(rand >= 95):

            raw_X_test.append(series_to_arr(raw_series))
            smoothed_X_test.append(series_to_arr(smoothed_series))
            translate_X_test.append(series_to_arr(translate_series))
            final_X_test.append(series_to_arr(final_series))

            o_label_test.append(int(file[4]))
            f_label_test.append(int(file[7]))
            b_label_test.append(int(file[10]))
            s_label_test.append(int(file[13]))
            l_label_test.append(int(file[16]))


print(len(raw_X_train))
print(len(raw_X_test))

raw_X_train = to_time_series_dataset(raw_X_train)
raw_X_test = to_time_series_dataset(raw_X_test)

smoothed_X_train = to_time_series_dataset(smoothed_X_train)
smoothed_X_test = to_time_series_dataset(smoothed_X_test)

translate_X_train = to_time_series_dataset(translate_X_train)
translate_X_test = to_time_series_dataset(translate_X_test)

final_X_train = to_time_series_dataset(final_X_train)
final_X_test = to_time_series_dataset(final_X_test)

0
26
28
30
28
27
24
22
32
32
33
16
22
24
34
36
20
19
35
31
38
30
28
29
23
28
35
22
28
439
25


In [45]:
# finding optimal k (DO NOT RUN)

'''o_err = []
f_err = []
b_err = []
s_err = []
l_err = []

print(np.arange(1, 20))

for n in range(1, 10):
    oknn = KNeighborsTimeSeriesClassifier(n_neighbors=n)
    oknn.fit(X_train, o_label_train)
    fknn = KNeighborsTimeSeriesClassifier(n_neighbors=n)
    fknn.fit(X_train, f_label_train)
    bknn = KNeighborsTimeSeriesClassifier(n_neighbors=n)
    bknn.fit(X_train, b_label_train)
    sknn = KNeighborsTimeSeriesClassifier(n_neighbors=n)
    sknn.fit(X_train, s_label_train)
    lknn = KNeighborsTimeSeriesClassifier(n_neighbors=n)
    lknn.fit(X_train, l_label_train)

    o_err.append(E2(oknn.predict(X_test), o_label_test))
    f_err.append(E2(fknn.predict(X_test), f_label_test))
    b_err.append(E2(bknn.predict(X_test), b_label_test))
    s_err.append(E2(sknn.predict(X_test), s_label_test))
    l_err.append(E2(lknn.predict(X_test), l_label_test))
'''

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


In [ ]:
# visualizing errors (DO NOT RUN)
fig, ax = plt.subplots(5, figsize=(20, 25))
ax[0].set_title('overstride')
ax[1].set_title('foward lean')
ax[2].set_title('backward lean')
ax[3].set_title('sweep')
ax[4].set_title('low arms')

print(o_err)
print(f_err)
print(b_err)
print(s_err)
print(l_err)


ax[0].scatter(np.arange(1, 10), o_err)
ax[1].scatter(np.arange(1, 10), f_err)
ax[2].scatter(np.arange(1, 10), b_err)
ax[3].scatter(np.arange(1, 10), s_err)
ax[4].scatter(np.arange(1, 10), l_err)

In [ ]:
# checking accuracies
oknn = KNeighborsTimeSeriesClassifier(n_neighbors=6)
oknn.fit(raw_X_train, o_label_train)
fknn = KNeighborsTimeSeriesClassifier(n_neighbors=4)
fknn.fit(raw_X_train, f_label_train)
bknn = KNeighborsTimeSeriesClassifier(n_neighbors=4)
bknn.fit(raw_X_train, b_label_train)
sknn = KNeighborsTimeSeriesClassifier(n_neighbors=6)
sknn.fit(raw_X_train, s_label_train)
lknn = KNeighborsTimeSeriesClassifier(n_neighbors=9)
lknn.fit(raw_X_train, l_label_train)

oknn_pred = oknn.predict(raw_X_test)
fknn_pred = fknn.predict(raw_X_test)
bknn_pred = bknn.predict(raw_X_test)
sknn_pred = sknn.predict(raw_X_test)
lknn_pred = lknn.predict(raw_X_test)

print(str(E0(oknn_pred, o_label_test)))
print(str(E1(oknn_pred, o_label_test)))
print(str(E2(oknn_pred, o_label_test)))
print(str(E3(oknn_pred, o_label_test)))

print(str(E0(fknn_pred, f_label_test)))
print(str(E1(fknn_pred, f_label_test)))
print(str(E2(fknn_pred, f_label_test)))
print(str(E3(fknn_pred, f_label_test)))

print(str(E0(bknn_pred, b_label_test)))
print(str(E1(bknn_pred, b_label_test)))
print(str(E2(bknn_pred, b_label_test)))
print(str(E3(bknn_pred, b_label_test)))

print(str(E0(sknn_pred, s_label_test)))
print(str(E1(sknn_pred, s_label_test)))
print(str(E2(sknn_pred, s_label_test)))
print(str(E3(sknn_pred, s_label_test)))

print(str(E0(lknn_pred, l_label_test)))
print(str(E1(lknn_pred, l_label_test)))
print(str(E2(lknn_pred, l_label_test)))
print(str(E3(lknn_pred, l_label_test)))


print('\n\n\n\n')

oknn = KNeighborsTimeSeriesClassifier(n_neighbors=6)
fknn = KNeighborsTimeSeriesClassifier(n_neighbors=4)
bknn = KNeighborsTimeSeriesClassifier(n_neighbors=4)
sknn = KNeighborsTimeSeriesClassifier(n_neighbors=6)
lknn = KNeighborsTimeSeriesClassifier(n_neighbors=9)

oknn.fit(smoothed_X_train, o_label_train)
fknn.fit(smoothed_X_train, f_label_train)
bknn.fit(smoothed_X_train, b_label_train)
sknn.fit(smoothed_X_train, s_label_train)
lknn.fit(smoothed_X_train, l_label_train)

oknn_pred = oknn.predict(smoothed_X_test)
fknn_pred = fknn.predict(smoothed_X_test)
bknn_pred = bknn.predict(smoothed_X_test)
sknn_pred = sknn.predict(smoothed_X_test)
lknn_pred = lknn.predict(smoothed_X_test)

print(str(E0(oknn_pred, o_label_test)))
print(str(E1(oknn_pred, o_label_test)))
print(str(E2(oknn_pred, o_label_test)))
print(str(E3(oknn_pred, o_label_test)))

print(str(E0(fknn_pred, f_label_test)))
print(str(E1(fknn_pred, f_label_test)))
print(str(E2(fknn_pred, f_label_test)))
print(str(E3(fknn_pred, f_label_test)))

print(str(E0(bknn_pred, b_label_test)))
print(str(E1(bknn_pred, b_label_test)))
print(str(E2(bknn_pred, b_label_test)))
print(str(E3(bknn_pred, b_label_test)))

print(str(E0(sknn_pred, s_label_test)))
print(str(E1(sknn_pred, s_label_test)))
print(str(E2(sknn_pred, s_label_test)))
print(str(E3(sknn_pred, s_label_test)))

print(str(E0(lknn_pred, l_label_test)))
print(str(E1(lknn_pred, l_label_test)))
print(str(E2(lknn_pred, l_label_test)))
print(str(E3(lknn_pred, l_label_test)))



print('\n\n\n\n')

oknn = KNeighborsTimeSeriesClassifier(n_neighbors=6)
fknn = KNeighborsTimeSeriesClassifier(n_neighbors=4)
bknn = KNeighborsTimeSeriesClassifier(n_neighbors=4)
sknn = KNeighborsTimeSeriesClassifier(n_neighbors=6)
lknn = KNeighborsTimeSeriesClassifier(n_neighbors=9)

oknn.fit(translate_X_train, o_label_train)
fknn.fit(translate_X_train, f_label_train)
bknn.fit(translate_X_train, b_label_train)
sknn.fit(translate_X_train, s_label_train)
lknn.fit(translate_X_train, l_label_train)

oknn_pred = oknn.predict(translate_X_test)
fknn_pred = fknn.predict(translate_X_test)
bknn_pred = bknn.predict(translate_X_test)
sknn_pred = sknn.predict(translate_X_test)
lknn_pred = lknn.predict(translate_X_test)

print(str(E0(oknn_pred, o_label_test)))
print(str(E1(oknn_pred, o_label_test)))
print(str(E2(oknn_pred, o_label_test)))
print(str(E3(oknn_pred, o_label_test)))

print(str(E0(fknn_pred, f_label_test)))
print(str(E1(fknn_pred, f_label_test)))
print(str(E2(fknn_pred, f_label_test)))
print(str(E3(fknn_pred, f_label_test)))

print(str(E0(bknn_pred, b_label_test)))
print(str(E1(bknn_pred, b_label_test)))
print(str(E2(bknn_pred, b_label_test)))
print(str(E3(bknn_pred, b_label_test)))

print(str(E0(sknn_pred, s_label_test)))
print(str(E1(sknn_pred, s_label_test)))
print(str(E2(sknn_pred, s_label_test)))
print(str(E3(sknn_pred, s_label_test)))

print(str(E0(lknn_pred, l_label_test)))
print(str(E1(lknn_pred, l_label_test)))
print(str(E2(lknn_pred, l_label_test)))
print(str(E3(lknn_pred, l_label_test)))




print('\n\n\n\n')

oknn = KNeighborsTimeSeriesClassifier(n_neighbors=6)
fknn = KNeighborsTimeSeriesClassifier(n_neighbors=4)
bknn = KNeighborsTimeSeriesClassifier(n_neighbors=4)
sknn = KNeighborsTimeSeriesClassifier(n_neighbors=6)
lknn = KNeighborsTimeSeriesClassifier(n_neighbors=9)

oknn.fit(final_X_train, o_label_train)
fknn.fit(final_X_train, f_label_train)
bknn.fit(final_X_train, b_label_train)
sknn.fit(final_X_train, s_label_train)
lknn.fit(final_X_train, l_label_train)

oknn_pred = oknn.predict(final_X_test)
fknn_pred = fknn.predict(final_X_test)
bknn_pred = bknn.predict(final_X_test)
sknn_pred = sknn.predict(final_X_test)
lknn_pred = lknn.predict(final_X_test)

print(str(E0(oknn_pred, o_label_test)))
print(str(E1(oknn_pred, o_label_test)))
print(str(E2(oknn_pred, o_label_test)))
print(str(E3(oknn_pred, o_label_test)))

print(str(E0(fknn_pred, f_label_test)))
print(str(E1(fknn_pred, f_label_test)))
print(str(E2(fknn_pred, f_label_test)))
print(str(E3(fknn_pred, f_label_test)))

print(str(E0(bknn_pred, b_label_test)))
print(str(E1(bknn_pred, b_label_test)))
print(str(E2(bknn_pred, b_label_test)))
print(str(E3(bknn_pred, b_label_test)))

print(str(E0(sknn_pred, s_label_test)))
print(str(E1(sknn_pred, s_label_test)))
print(str(E2(sknn_pred, s_label_test)))
print(str(E3(sknn_pred, s_label_test)))

print(str(E0(lknn_pred, l_label_test)))
print(str(E1(lknn_pred, l_label_test)))
print(str(E2(lknn_pred, l_label_test)))
print(str(E3(lknn_pred, l_label_test)))